# Stock purchase recommendations with Machine Learning

In [272]:
import pandas_datareader.data as web
import pandas as pd
import numpy as np
import talib as ta
import matplotlib.pyplot as plt
from tqdm import tqdm # progress bar

In [118]:
# read in the Quandl.com API key - saved in separate file to keep it private

filepath = 'confidential-API-key.txt'  
with open(filepath) as fp:  
   my_Quandl_API_key = fp.readline()

## Load stock data from Quandl

In [119]:
def load_stock_data(ticker, start_date, end_date, Quandl_API_key=my_Quandl_API_key):
    '''
    Downloads stock data from Quandl, drop some columns, resort datafram, and return
    Inputs:
        ticker - a stock ticker symbol (needs to be valid - no error checking implemented)
        start_date - first date of stock prices
        end_date - last date of stock prices
        Quandl_API_key - string with valid API key for Quandl.com data queries
    Outputs:
        stock_data - DataFrame with stock price data, sorted in ascending date order
    '''
    
    # download data from Quandl with Pandas Datareader
    stock_data = web.DataReader(name=symbol, data_source='quandl', start=start_date, end=end_date, access_key=Quandl_API_key)
    
    # need ascending index for the TA-lib indicators to work properly
    stock_data.sort_index(inplace=True)
    
    # keep only the columns with adjusted data to eliminate any issues due to stock splits
    stock_data = stock_data[['AdjVolume', 'AdjOpen', 'AdjHigh', 'AdjLow', 'AdjClose']]
    
    # DataFram has a second level column index with the stock ticker - not needed so drop it
    stock_data.columns = stock_data.columns.droplevel(1)
    
    # add row index (highest number is most recent date) - this will be used to later restack the rows for feature matrix
    stock_data['row_index'] = range(0, stock_data.shape[0])
    
    return stock_data    

In [120]:
# test loading stock data:

start = '2015-04-22'
end = '2017-04-22'
symbol = ['AAPL']

df = load_stock_data(symbol, start, end)

df.head()

Attributes,AdjVolume,AdjOpen,AdjHigh,AdjLow,AdjClose,row_index
Date,,,,,,
2015-04-22,37654505.0,121.113778,122.906785,120.474781,122.668353,0
2015-04-23,45770902.0,122.363160,124.385061,122.210564,123.669766,1
2015-04-24,44525905.0,124.451822,124.585344,123.250126,124.251540,2
2015-04-27,96954207.0,126.187605,126.969661,125.081282,126.511872,3
2015-04-28,118923970.0,128.233349,128.314416,123.574394,124.518583,4


In [121]:
def add_TALib_indicator(df, attribute, indicator_func, *args):
    '''
    Adds a column to a dataframe:
        column name is the name of the technical indicator as specified by indicator_func
        column content is the function calculated on the attribute column
    Example: add_TALib_indicator(df, 'AdjClose', ta.RSI, 14) creates a new column called RSI with 
             the 14 day RSI of the values of the column 'AdjClose'
    Inputs:
        df - dataframe - needs to be sorted in date ascending order
        attribute - column name to be used in TA-Lib calculation
        indicator_func - name of a TA-Lib function
        *args - optional parameters for indicator_func
        
    Oupputs:
        df - datarame with new column added
        func_name - name of the new colunm
    
    '''
    # get the name of the indicator from TA-Lib
    func_name = attribute + indicator_func.__name__
    
    # add new column, calculated based on attribute column
    df.loc[:, func_name] = indicator_func(df.loc[:, attribute].values, *args)
    
    return df, func_name

In [122]:
df, indicator_name = add_TALib_indicator(df, 'AdjClose', ta.RSI, 14)

df.tail()

Attributes,AdjVolume,AdjOpen,AdjHigh,AdjLow,AdjClose,row_index,AdjCloseRSI
Date,,,,,,,
2017-04-17,16582094.0,140.333992,140.730752,139.728933,140.681157,500,54.018130
2017-04-18,14697544.0,140.264559,140.889456,139.966989,140.056260,501,50.438090
2017-04-19,17328375.0,140.730752,140.849780,139.312335,139.540472,502,47.632042
2017-04-20,23319562.0,140.076098,141.762328,140.016584,141.286216,503,56.460990
2017-04-21,17320928.0,141.286216,141.524272,140.700995,141.117593,504,55.487879


In [123]:
df.head(16)
#confirms NaN for RSI on top for first 14 dates since it is a 14 day RSI

Attributes,AdjVolume,AdjOpen,AdjHigh,AdjLow,AdjClose,row_index,AdjCloseRSI
Date,,,,,,,
2015-04-22,37654505.0,121.113778,122.906785,120.474781,122.668353,0,NaN
2015-04-23,45770902.0,122.363160,124.385061,122.210564,123.669766,1,NaN
2015-04-24,44525905.0,124.451822,124.585344,123.250126,124.251540,2,NaN
2015-04-27,96954207.0,126.187605,126.969661,125.081282,126.511872,3,NaN
2015-04-28,118923970.0,128.233349,128.314416,123.574394,124.518583,4,NaN
2015-04-29,63386083.0,124.137092,125.500922,122.363160,122.687428,5,NaN
2015-04-30,83195423.0,121.600179,121.962595,118.815296,119.358921,6,NaN
2015-05-01,58512638.0,120.264961,124.108481,119.501980,122.983083,7,NaN
2015-05-04,50988278.0,123.507633,124.528120,122.325011,122.744651,8,NaN


## Define Target column: profitability of trade

The goal is the define a column "setup_for_profitable_trade" based on a trading strategy. The data will be analyzed for this trade setup:

Run the analysis in the evening after market close. That day's row of market data will then be either deemed as profitable or not for a trade to be initiated and closed in the following days. This setup of the data ensures no lookahead is happening.

## Trading Strategy:

1. After market close on day N decide if buying stock at market Open on day N+1
2. Open position: submit market order for day N+1 prior to market open
3. Close position: submit sell order for market open for day N+2

Threshold to decide to buy the stock: expected profit from open to close: 0.5% profit

Store in row N if this trade was deemed profitable.

In [124]:
def add_flag_for_profitable_trade_setups(df, delete_interim_calculation_cols=True):
    '''
    Adds column to dataframe that identifies profitable trading setup
    Inputs:
        df - dataframe with stock data
        delete_interim_calculation_cols - if false: keep the interim calculations - good for debugging
    Outputs:
        df - dataframe with additional columns
    
    '''
    
    # trade strategy: after market close on day N, set a buy at market open on day N+1 and sell at market close on day N+1
    # for expected gain of at least 0.5%
    profitability_threshold = 0.005

    # use helper columns to calculate profit
    df['strategy_open_price'] = df['AdjOpen'].shift(-1) # AdjOpen from day N+1
    df['strategy_close_price'] = df['AdjOpen'].shift(-2) # AdjOpen from day N+2
    df['strategy_profit_dollars'] = df['strategy_close_price'] - df['strategy_open_price']
    df['strategy_profitability'] = df['strategy_profit_dollars'] / df['strategy_open_price']

    # use categorical field to encode "setup_for_succesful_trade": 1=yes, 0=no
    df['setup_for_profitable_trade'] = df['strategy_profitability'] >= profitability_threshold
    
    if delete_interim_calculation_cols:
        df = df.drop(columns=['strategy_open_price', 'strategy_close_price', 'strategy_profit_dollars', 'strategy_profitability'])
    
    return df

In [127]:
# testing: show calculations:
df = add_flag_for_profitable_trade_setups(df, False)
df.tail(10)

Attributes,AdjVolume,AdjOpen,AdjHigh,AdjLow,AdjClose,row_index,AdjCloseRSI,setup_for_profitable_trade,strategy_open_price,strategy_close_price,strategy_profit_dollars,strategy_profitability
Date,,,,,,,,,,,,
2017-04-07,16658543.0,142.565767,143.012122,142.109493,142.178926,495,64.426231,False,142.436820,141.782166,-0.654654,-0.004596
2017-04-10,18933397.0,142.436820,142.713758,141.742490,142.010303,496,63.211365,False,141.782166,140.453020,-1.329146,-0.009375
2017-04-11,30379376.0,141.782166,142.188845,138.925494,140.482777,497,53.389801,False,140.453020,140.760509,0.307489,0.002189
2017-04-12,20350000.0,140.453020,140.998565,139.867799,140.651400,498,54.235139,False,140.760509,140.333992,-0.426517,-0.003030
2017-04-13,17822880.0,140.760509,141.226702,139.907475,139.907475,499,49.932546,False,140.333992,140.264559,-0.069433,-0.000495
2017-04-17,16582094.0,140.333992,140.730752,139.728933,140.681157,500,54.018130,False,140.264559,140.730752,0.466193,0.003324
2017-04-18,14697544.0,140.264559,140.889456,139.966989,140.056260,501,50.438090,False,140.730752,140.076098,-0.654654,-0.004652
2017-04-19,17328375.0,140.730752,140.849780,139.312335,139.540472,502,47.632042,True,140.076098,141.286216,1.210118,0.008639
2017-04-20,23319562.0,140.076098,141.762328,140.016584,141.286216,503,56.460990,False,141.286216,NaN,NaN,NaN


In [128]:
# without verbose results:
df = add_flag_for_profitable_trade_setups(df)
df.tail()

Attributes,AdjVolume,AdjOpen,AdjHigh,AdjLow,AdjClose,row_index,AdjCloseRSI,setup_for_profitable_trade
Date,,,,,,,,
2017-04-17,16582094.0,140.333992,140.730752,139.728933,140.681157,500,54.018130,False
2017-04-18,14697544.0,140.264559,140.889456,139.966989,140.056260,501,50.438090,False
2017-04-19,17328375.0,140.730752,140.849780,139.312335,139.540472,502,47.632042,True
2017-04-20,23319562.0,140.076098,141.762328,140.016584,141.286216,503,56.460990,False
2017-04-21,17320928.0,141.286216,141.524272,140.700995,141.117593,504,55.487879,False


In [86]:
df['setup_for_profitable_trade'].sum()

181

In [135]:
# parameter for how many days of history to include in feature list
n_days_features = 5

# total lenght of df
n_data_points = df.shape[0]

### --- build pieces for the dataframe section unpivoting ---
Goal: pull out n_features rows, flatten, and build meaningful column names that indicate how many days' back the data is from

In [239]:
#split df into features and target (column setup_for_profitable_trade)
df_X_base_data = df.drop(columns=['setup_for_profitable_trade'])

In [234]:
i = 505
# pull out n_days_features of rows from current position
df_extract = df_X_base_data.iloc[i-n_days_features:i, :].copy()

# change the index the be "days into the past" - eg current day is 0, prior day is -1, ...
df_extract.loc[:, 'row_index'] = range(-n_days_features+1, 1)

# make this the new index
df_extract.set_index('row_index', inplace=True)

df_extract

Attributes,AdjVolume,AdjOpen,AdjHigh,AdjLow,AdjClose,AdjCloseRSI
row_index,,,,,,
-4,16582094.0,140.333992,140.730752,139.728933,140.681157,54.018130
-3,14697544.0,140.264559,140.889456,139.966989,140.056260,50.438090
-2,17328375.0,140.730752,140.849780,139.312335,139.540472,47.632042
-1,23319562.0,140.076098,141.762328,140.016584,141.286216,56.460990
0,17320928.0,141.286216,141.524272,140.700995,141.117593,55.487879


In [235]:
# unstack and make it tall (ie unpivot)
df_extract = df_extract.unstack().reset_index()
df_extract

,Attributes,row_index,0
0,AdjVolume,-4,1.658209e+07
1,AdjVolume,-3,1.469754e+07
2,AdjVolume,-2,1.732838e+07
3,AdjVolume,-1,2.331956e+07
4,AdjVolume,0,1.732093e+07
5,AdjOpen,-4,1.403340e+02
6,AdjOpen,-3,1.402646e+02
7,AdjOpen,-2,1.407308e+02
8,AdjOpen,-1,1.400761e+02
9,AdjOpen,0,1.412862e+02


In [236]:
# create new column with combined field names of attribute and index
# eg: AdjClose-1 for the adjusted close of day N-1 or AdjHigh-4 for the adjusted High of day N-4
df_extract['Attribute-index'] = df_extract['Attributes'] + df_extract['row_index'].apply(str)
# then drop Attributes and row_index columns since they are not needed anymore
df_extract.drop(columns=['Attributes', 'row_index'], inplace=True)
df_extract

,0,Attribute-index
0,1.658209e+07,AdjVolume-4
1,1.469754e+07,AdjVolume-3
2,1.732838e+07,AdjVolume-2
3,2.331956e+07,AdjVolume-1
4,1.732093e+07,AdjVolume0
5,1.403340e+02,AdjOpen-4
6,1.402646e+02,AdjOpen-3
7,1.407308e+02,AdjOpen-2
8,1.400761e+02,AdjOpen-1
9,1.412862e+02,AdjOpen0


In [256]:
# set index one and transpose
target_row = df_extract.set_index('Attribute-index').T
# we now have one row of data that represents the prior n_feature_days worth of data:
target_row

Attribute-index,AdjVolume-4,AdjVolume-3,AdjVolume-2,AdjVolume-1,AdjVolume0,AdjOpen-4,AdjOpen-3,AdjOpen-2,AdjOpen-1,AdjOpen0,...,AdjClose-4,AdjClose-3,AdjClose-2,AdjClose-1,AdjClose0,AdjCloseRSI-4,AdjCloseRSI-3,AdjCloseRSI-2,AdjCloseRSI-1,AdjCloseRSI0
0,16582094.0,14697544.0,17328375.0,23319562.0,17320928.0,140.333992,140.264559,140.730752,140.076098,141.286216,...,140.681157,140.05626,139.540472,141.286216,141.117593,54.01813,50.43809,47.632042,56.46099,55.487879


In [258]:
df_X_base_data.index.to_list()[504]

Timestamp('2017-04-21 00:00:00')

In [259]:
target_row['Index'] = df_X_base_data.index.to_list()[504]
target_row = target_row.set_index('Index')
target_row

Attribute-index,AdjVolume-4,AdjVolume-3,AdjVolume-2,AdjVolume-1,AdjVolume0,AdjOpen-4,AdjOpen-3,AdjOpen-2,AdjOpen-1,AdjOpen0,...,AdjClose-4,AdjClose-3,AdjClose-2,AdjClose-1,AdjClose0,AdjCloseRSI-4,AdjCloseRSI-3,AdjCloseRSI-2,AdjCloseRSI-1,AdjCloseRSI0
Index,,,,,,,,,,,,,,,,,,,,,
2017-04-21,16582094.0,14697544.0,17328375.0,23319562.0,17320928.0,140.333992,140.264559,140.730752,140.076098,141.286216,...,140.681157,140.05626,139.540472,141.286216,141.117593,54.01813,50.43809,47.632042,56.46099,55.487879


### --- done building the pieces, now implement in one loop to cycle through all rows of df ---

In [275]:
df_X = pd.DataFrame()

#split df into features and target (column setup_for_profitable_trade)
df_X_base_data = df.drop(columns=['setup_for_profitable_trade'])
df_y = df['setup_for_profitable_trade']

# cycle through each row of df, start at n_days_features-1 because we wouldn't have enough history for first rows
for i in tqdm(range(n_days_features, n_data_points+1), desc='reshaping data into feature rows'):
    # i contains the rows number of df
    
    df_extract = df.iloc[i-n_days_features:i, :]
    
    # pull out n_days_features of rows from current position
    df_extract = df_X_base_data.iloc[i-n_days_features:i, :].copy()

    # change the index the be "days into the past" - eg current day is 0, prior day is -1, ...
    df_extract.loc[:, 'row_index'] = range(-n_days_features+1, 1)

    # make this the new index
    df_extract.set_index('row_index', inplace=True)

    # unstack and make it tall (ie unpivot)
    df_extract = df_extract.unstack().reset_index()
    
    # create new column with combined field names of attribute and index
    # eg: AdjClose-1 for the adjusted close of day N-1 or AdjHigh-4 for the adjusted High of day N-4
    df_extract['Attribute-index'] = df_extract['Attributes'] + df_extract['row_index'].apply(str)
    # then drop Attributes and row_index columns since they are not needed anymore
    df_extract.drop(columns=['Attributes', 'row_index'], inplace=True)
    
    # set index one and transpose
    target_row = df_extract.set_index('Attribute-index').T
    # we now have one row of data that represents the prior n_feature_days worth of data
    
    # fill in the target_row index with the date from the index of the source dataframe df_X_base_data (ie, day N)
    target_row['Index'] = df_X_base_data.index.to_list()[i-1] # zero-indexed so need minus 1
    target_row = target_row.set_index('Index')

    df_X = df_X.append(target_row)
    
df_X.tail()

reshaping data into feature rows: 100%|██████| 501/501 [00:06<00:00, 72.71it/s]


Attribute-index,AdjVolume-4,AdjVolume-3,AdjVolume-2,AdjVolume-1,AdjVolume0,AdjOpen-4,AdjOpen-3,AdjOpen-2,AdjOpen-1,AdjOpen0,...,AdjClose-4,AdjClose-3,AdjClose-2,AdjClose-1,AdjClose0,AdjCloseRSI-4,AdjCloseRSI-3,AdjCloseRSI-2,AdjCloseRSI-1,AdjCloseRSI0
Index,,,,,,,,,,,,,,,,,,,,,
2017-04-17,18933397.0,30379376.0,20350000.0,17822880.0,16582094.0,142.436820,141.782166,140.453020,140.760509,140.333992,...,142.010303,140.482777,140.651400,139.907475,140.681157,63.211365,53.389801,54.235139,49.932546,54.018130
2017-04-18,30379376.0,20350000.0,17822880.0,16582094.0,14697544.0,141.782166,140.453020,140.760509,140.333992,140.264559,...,140.482777,140.651400,139.907475,140.681157,140.056260,53.389801,54.235139,49.932546,54.018130,50.438090
2017-04-19,20350000.0,17822880.0,16582094.0,14697544.0,17328375.0,140.453020,140.760509,140.333992,140.264559,140.730752,...,140.651400,139.907475,140.681157,140.056260,139.540472,54.235139,49.932546,54.018130,50.438090,47.632042
2017-04-20,17822880.0,16582094.0,14697544.0,17328375.0,23319562.0,140.760509,140.333992,140.264559,140.730752,140.076098,...,139.907475,140.681157,140.056260,139.540472,141.286216,49.932546,54.018130,50.438090,47.632042,56.460990
2017-04-21,16582094.0,14697544.0,17328375.0,23319562.0,17320928.0,140.333992,140.264559,140.730752,140.076098,141.286216,...,140.681157,140.056260,139.540472,141.286216,141.117593,54.018130,50.438090,47.632042,56.460990,55.487879


### indexing testing around: works for one column:

In [104]:
price['Open']['AAPL'].values[-20:]

array([129.38, 129.08, 129.46, 129.  , 128.75, 129.39, 129.5 , 129.14,
       129.89, 130.31, 130.02, 129.95, 130.62, 131.31, 131.  , 131.25,
       132.01, 132.64, 132.65, 133.55])

In [117]:
ta.RSI(price['Open']['AAPL'].values, 14)[-20:]

array([64.67918491, 61.52503529, 63.92482519, 59.11784324, 56.62564291,
       61.14182474, 61.87655495, 58.01067873, 63.17286007, 65.71484641,
       62.50662269, 61.72331015, 66.10200343, 69.91852913, 66.30652914,
       67.75331269, 71.72775641, 74.53015083, 74.57323366, 78.15471567])

In [129]:
price.loc[:, idx[('RSI'), ('AAPL')]] = ta.RSI(price.loc[:, idx[('Open'), ('AAPL')]].values, 14)
price['RSI']['AAPL'].tail(20)

Date
2017-03-24    64.679185
2017-03-27    61.525035
2017-03-28    63.924825
2017-03-29    59.117843
2017-03-30    56.625643
2017-03-31    61.141825
2017-04-03    61.876555
2017-04-04    58.010679
2017-04-05    63.172860
2017-04-06    65.714846
2017-04-07    62.506623
2017-04-10    61.723310
2017-04-11    66.102003
2017-04-12    69.918529
2017-04-13    66.306529
2017-04-17    67.753313
2017-04-18    71.727756
2017-04-19    74.530151
2017-04-20    74.573234
2017-04-21    78.154716
Name: AAPL, dtype: float64

In [116]:
idx = pd.IndexSlice
price.loc[:, idx[:, ('AAPL','MSFT')]].head()

Attributes   Open          High            Low          Close         \
Symbols      AAPL   MSFT   AAPL   MSFT    AAPL    MSFT   AAPL   MSFT   
Date                                                                   
2015-04-22  97.00  97.00  99.35  99.35  96.240  96.240  97.84  97.84   
2015-04-23  97.44  97.44  97.52  97.52  96.555  96.555  97.00  97.00   
2015-04-24  96.99  96.99  99.08  99.08  96.840  96.840  98.74  98.74   
2015-04-27  98.74  98.74  98.94  98.94  96.260  96.260  96.44  96.44   
2015-04-28  96.27  96.27  96.89  96.89  95.780  95.780  96.83  96.83   

Attributes      Volume                 ...        AdjOpen             \
Symbols           AAPL        MSFT     ...           AAPL       MSFT   
Date                                   ...                             
2015-04-22  19253001.0  19253001.0     ...      89.985614  89.985614   
2015-04-23   6317607.0   6317607.0     ...      90.393796  90.393796   
2015-04-24   7741339.0   7741339.0     ...      89.976337  89.976337   
2015-04-27   7203201.0   7203201.0     ...      91.599789  91.599789   
2015-04-28   4357219.0   4357219.0     ...      89.308403  89.308403   

Attributes    AdjHigh                AdjLow              AdjClose             \
Symbols          AAPL       MSFT       AAPL       MSFT       AAPL       MSFT   
Date                                                                           
2015-04-22  92.165678  92.165678  89.280572  89.280572  90.764871  90.764871   
2015-04-23  90.468011  90.468011  89.572793  89.572793  89.985614  89.985614   
2015-04-24  91.915202  91.915202  89.837184  89.837184  91.599789  91.599789   
2015-04-27  91.785326  91.785326  89.299126  89.299126  89.466109  89.466109   
2015-04-28  89.883568  89.883568  88.853836  88.853836  89.827907  89.827907   

Attributes   AdjVolume              
Symbols           AAPL        MSFT  
Date                                
2015-04-22  19253001.0  19253001.0  
2015-04-23   6317607.0   6317607.0  
2015-04-24   7741339.0   7741339.0  
2015-04-27   7203201.0   7203201.0  
2015-04-28   4357219.0   4357219.0  

[5 rows x 24 columns]

In [80]:
price.loc[:, idx[('AdjOpen','AdjClose'), ('AAPL','MSFT')]].head()

Attributes    AdjOpen              AdjClose           
Symbols          AAPL       MSFT       AAPL       MSFT
Date                                                  
2015-04-22  89.985614  89.985614  90.764871  90.764871
2015-04-23  90.393796  90.393796  89.985614  89.985614
2015-04-24  89.976337  89.976337  91.599789  91.599789
2015-04-27  91.599789  91.599789  89.466109  89.466109
2015-04-28  89.308403  89.308403  89.827907  89.827907

In [36]:
price.loc[:, idx[('AdjClose'), ('AAPL')]].head()

NameError: name 'idx' is not defined